In [196]:
import cv2  
import numpy as np
import matplotlib.pyplot as plot
import time

### 初始化，打开摄像头等待用户选择出矩形框

In [197]:
# 定义鼠标回调函数
def myMouse(event,x,y,flags,param):
  global flag,frame,start_flag,start_point,end_point,frame_copy1,frame_copy2,min_x,min_y,width,height,image_target
  if(event == cv2.EVENT_LBUTTONDOWN):
    start_flag = 1
    start_point = (x,y)
  if(event == cv2.EVENT_MOUSEMOVE and start_flag==1):
    frame_copy1 = frame.copy()
    end_point = (x,y)
    if(start_point != end_point):
      cv2.rectangle(frame_copy1,start_point,end_point,(255,0,0),3)
      cv2.imshow('get_target',frame_copy1)
  if(event == cv2.EVENT_LBUTTONUP):
    frame_copy2 = frame.copy()
    start_flag = 0
    flag = 1
    start_x,start_y = start_point
    min_x = min(start_x,x)     
    min_y = min(start_y,y)
    width = abs(x - start_x)
    height = abs(y - start_y)
    image_target = frame_copy2[min_y:min_y+height, min_x:min_x+width]
    cv2.imshow("target",image_target)
    

In [198]:
# cap = cv2.VideoCapture(0)
# flag=0
# start_flag = 0
# cv2.namedWindow('get_target')
# cv2.setMouseCallback('get_target',myMouse)
# while(1):
#   if(start_flag == 0):
#     ret,frame = cap.read()#摄像头读取，ret为是否程庚打开摄像头，frame是每一帧图像
#     frame = cv2.flip(frame, 1)#摄像头是和人对立的，将图像左右调换回来正常显示
#   cv2.imshow("get_target",frame)
#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break
#   if(flag):break
# cap.release()
# cv2.destroyAllWindows()

### 再次打开摄像头开始追踪

In [199]:
def match0_sift():
  global frame_copy2,rect,mask_ouput,start_point,end_point
  if(f==0):
    pre_gray = cv2.cvtColor(image_target,cv2.COLOR_RGB2GRAY)
    pre_gray = cv2.GaussianBlur(pre_gray,(7,7),0)
    sift = cv2.xfeatures2d_SIFT.create()  # 新建一个sift算子
    keyPoint, descriptor = sift.detectAndCompute(pre_gray, None)
    # print(keyPoint,descriptor)
    # arr= []
    # for i in keyPoint:
    #   # if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
    #     arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
    # pre_corners = np.array(arr)
  frame_gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
  frame_gray = cv2.GaussianBlur(frame_gray,(7,7),0)
  keyPoint_cur,descriptor_cur = sift.detectAndCompute(frame_gray,None)
  # arr= []
  # for i in keyPoint_cur:
  #     # if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
  #       arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
  # cur_corners = np.array(arr)
  bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)  # 新建BFMatcher对象，sift的normType应该使用NORM_L2或者NORM_L1
  matches = bf.match(descriptor, descriptor_cur)
  matched = False
  matches = sorted(matches, key=lambda x: x.distance)
  # print(matches)
  arr_pre = []
  arr_cur = []
  for x in matches[:10]:
    arr_pre.append((keyPoint[x.queryIdx].pt[0]+min_x,keyPoint[x.queryIdx].pt[1]+min_y))
    arr_cur.append((keyPoint_cur[x.trainIdx].pt[0],keyPoint_cur[x.trainIdx].pt[1]))
  pre_corners = np.array(arr_pre)
  cur_corners = np.array(arr_cur)
  # H = cv2.findHomography(pre_corners,cur_corners)
  H = cv2.findHomography(pre_corners,cur_corners,method=cv2.RANSAC)
  res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
  rect = np.zeros(frame.shape,np.uint8)
  cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
  rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
  res = cv2.drawMatches(image_target, keyPoint,cv2.add(frame,rect), keyPoint_cur, matches[:30],frame,flags=2,matchColor=(0,255,0))
  cv2.imshow("result",res)
  # H = cv2.findHomography(pre_corners,cur_corners)
  # res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
  # rect = np.zeros(frame.shape,np.uint8)
  # cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
  # rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
  # res[0:frame.shape[0],0:frame.shape[1],:] = cv2.add(frame,rect)
  # res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
  # for i in range(cur_corners.shape[0]):
  #   cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(pre_corners[i][0])+frame.shape[1],int(pre_corners[i][1])),(0,255,0),1)
  # # print(index_delete)
  # cv2.imshow("result",res)

In [200]:
def match0_surf():
  global frame_copy2,rect,mask_ouput,start_point,end_point
  if(f==0):
    pre_gray = cv2.cvtColor(image_target,cv2.COLOR_RGB2GRAY)
    pre_gray = cv2.GaussianBlur(pre_gray,(7,7),0)
    surf = cv2.xfeatures2d_SURF.create()  # 新建一个surf算子
    keyPoint, descriptor = surf.detectAndCompute(pre_gray, None)
    # print(keyPoint,descriptor)
    # arr= []
    # for i in keyPoint:
    #   # if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
    #     arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
    # pre_corners = np.array(arr)
  frame_gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
  frame_gray = cv2.GaussianBlur(frame_gray,(7,7),0)
  keyPoint_cur,descriptor_cur = surf.detectAndCompute(frame_gray,None)
  # arr= []
  # for i in keyPoint_cur:
  #     # if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
  #       arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
  # cur_corners = np.array(arr)
  bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)  # 新建BFMatcher对象，sift的normType应该使用NORM_L2或者NORM_L1
  matches = bf.match(descriptor, descriptor_cur)
  matched = False
  if(len(matches)>50):
    matched = True
  matches = sorted(matches, key=lambda x: x.distance)
  # print(matches)
  arr_pre = []
  arr_cur = []
  for x in matches[:10]:
    arr_pre.append((keyPoint[x.queryIdx].pt[0]+min_x,keyPoint[x.queryIdx].pt[1]+min_y))
    arr_cur.append((keyPoint_cur[x.trainIdx].pt[0],keyPoint_cur[x.trainIdx].pt[1]))
  pre_corners = np.array(arr_pre)
  cur_corners = np.array(arr_cur)
  H = cv2.findHomography(pre_corners,cur_corners)
  res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
  rect = np.zeros(frame.shape,np.uint8)
  cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
  rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
  res = cv2.drawMatches(image_target, keyPoint,cv2.add(frame,rect), keyPoint_cur, matches[:30],frame,flags=2,matchColor=(0,255,0))
  cv2.imshow("result",res)
  # H = cv2.findHomography(pre_corners,cur_corners)
  # res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
  # rect = np.zeros(frame.shape,np.uint8)
  # cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
  # rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
  # res[0:frame.shape[0],0:frame.shape[1],:] = cv2.add(frame,rect)
  # res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
  # for i in range(cur_corners.shape[0]):
  #   cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(pre_corners[i][0])+frame.shape[1],int(pre_corners[i][1])),(0,255,0),1)
  # # print(index_delete)
  # cv2.imshow("result",res)

In [201]:
def match0_orb():
  global frame_copy2,rect,mask_ouput,start_point,end_point
  if(f==0):
    pre_gray = cv2.cvtColor(image_target,cv2.COLOR_RGB2GRAY)
    pre_gray = cv2.GaussianBlur(pre_gray,(7,7),0)
    orb = cv2.ORB_create() # 新建orb算子
    keyPoint, descriptor = orb.detectAndCompute(pre_gray, None)
    # print(keyPoint,descriptor)
    # arr= []
    # for i in keyPoint:
    #   # if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
    #     arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
    # pre_corners = np.array(arr)
  frame_gray = cv2.cvtColor(frame,cv2.COLOR_RGB2GRAY)
  frame_gray = cv2.GaussianBlur(frame_gray,(7,7),0)
  keyPoint_cur,descriptor_cur = orb.detectAndCompute(frame_gray,None)
  # arr= []
  # for i in keyPoint_cur:
  #     # if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
  #       arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
  # cur_corners = np.array(arr)
  bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)  # 新建BFMatcher对象，sift的normType应该使用NORM_L2或者NORM_L1
  matches = bf.match(descriptor, descriptor_cur)
  matched = False
  if(len(matches)>50):
    matched = True
  matches = sorted(matches, key=lambda x: x.distance)
  # print(matches)
  arr_pre = []
  arr_cur = []
  for x in matches[:10]:
    arr_pre.append((keyPoint[x.queryIdx].pt[0]+min_x,keyPoint[x.queryIdx].pt[1]+min_y))
    arr_cur.append((keyPoint_cur[x.trainIdx].pt[0],keyPoint_cur[x.trainIdx].pt[1]))
  pre_corners = np.array(arr_pre)
  cur_corners = np.array(arr_cur)
  H = cv2.findHomography(pre_corners,cur_corners)
  res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
  rect = np.zeros(frame.shape,np.uint8)
  cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
  rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
  res = cv2.drawMatches(image_target, keyPoint,cv2.add(frame,rect), keyPoint_cur, matches[:30],frame,flags=2,matchColor=(0,255,0))
  cv2.imshow("result",res)
  # H = cv2.findHomography(pre_corners,cur_corners)
  # res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
  # rect = np.zeros(frame.shape,np.uint8)
  # cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
  # rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
  # res[0:frame.shape[0],0:frame.shape[1],:] = cv2.add(frame,rect)
  # res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
  # for i in range(cur_corners.shape[0]):
  #   cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(pre_corners[i][0])+frame.shape[1],int(pre_corners[i][1])),(0,255,0),1)
  # # print(index_delete)
  # cv2.imshow("result",res)

In [202]:
def match():
  global frame_copy2,f,rect,mask_ouput,init_corners,pre_frame,pre_corners
  if(f==0):
    pre_gray = cv2.cvtColor(frame_copy2,cv2.COLOR_RGB2GRAY)
    #检测图像中的角点
    corners = cv2.goodFeaturesToTrack(pre_gray,50,0.01,20)
    corners = corners.reshape(-1,2)
    arr= []
    for i in corners:
      if(i[0]>=min_x and i[0]<=min_x+width and i[1]>=min_y and i[1]<=min_y+height):
        arr.append(i)
        # print(i)
    # print(arr)
    pre_corners = np.array(arr)
    init_corners = np.array(arr)
    # f=1
  cur_corners,status,err = cv2.calcOpticalFlowPyrLK(pre_frame,frame,pre_corners,None)
  pre_frame = frame.copy()
  tr_num = 0
  for i in status:
    if(i[0]==1):
      tr_num+=1
  # print(tr_num)
  if(tr_num >= 4):
    H = cv2.findHomography(pre_corners,cur_corners,cv2.RANSAC)
    pre_corners = cur_corners.copy()
    res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
    if(f==0):
      rect = np.zeros(frame.shape,np.uint8)
      cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
      rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
      f=1
    else:
      rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
    res[0:frame.shape[0],0:frame.shape[1],:] = cv2.add(frame,rect)
    res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
    index_delete = []
    for i in range(cur_corners.shape[0]):
      if(cur_corners[i][0]<0 or cur_corners[i][0]>frame.shape[1] or cur_corners[i][1]<0 or cur_corners[i][1]>frame.shape[0]):
        index_delete.append(i)
        continue
      cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(init_corners[i][0])+frame.shape[1],int(init_corners[i][1])),(0,255,0),1)
    # print(index_delete)
    cur_corners = np.delete(cur_corners,index_delete)
    cv2.imshow("result",res)

In [203]:
def match2():
  global frame_copy2,rect,mask_ouput,start_point,end_point
  pre_gray = cv2.cvtColor(frame_copy2,cv2.COLOR_RGB2GRAY)
  #检测图像中的角点
  corners = cv2.goodFeaturesToTrack(pre_gray,50,0.01,20)
  corners = corners.reshape(-1,2)
  arr= []
  for i in corners:
    if(i[0]>=min_x and i[0]<=min_x+width and i[1]>=min_y and i[1]<=min_y+height):
      arr.append(i)
      # print(i)
  # print(arr)
  pre_corners = np.array(arr)
  cur_corners,status,err = cv2.calcOpticalFlowPyrLK(frame_copy2,frame,pre_corners,None)
  tr_num = 0
  for i in status:
    if(i[0]==1):
      tr_num+=1
  # print(tr_num)
  if(tr_num >= 4):
    H = cv2.findHomography(pre_corners,cur_corners,cv2.RANSAC)
    res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
    rect = np.zeros(frame.shape,np.uint8)
    cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
    rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
    res[0:frame.shape[0],0:frame.shape[1],:] = cv2.add(frame,rect)
    res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
    for i in range(cur_corners.shape[0]):
      cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(pre_corners[i][0])+frame.shape[1],int(pre_corners[i][1])),(0,255,0),1)
    # print(index_delete)
    cv2.imshow("result",res)

In [204]:
def match2_sift():
  global frame_copy2,rect,mask_ouput,start_point,end_point
  pre_gray = cv2.cvtColor(frame_copy2,cv2.COLOR_RGB2GRAY)
  pre_gray = cv2.GaussianBlur(pre_gray,(7,7),0)
  sift = cv2.xfeatures2d_SIFT.create()  # 新建一个sift算子
  keyPoint, descriptor = sift.detectAndCompute(pre_gray, None)
  arr= []
  for i in keyPoint:
    if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
      arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
  pre_corners = np.array(arr)
  cur_corners,status,err = cv2.calcOpticalFlowPyrLK(frame_copy2,frame,pre_corners,None)
  tr_num = 0
  for i in status:
    if(i[0]==1):
      tr_num+=1
  # print(tr_num)
  if(tr_num >= 4):
    H = cv2.findHomography(pre_corners,cur_corners,method=cv2.RANSAC)
    res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
    rect = np.zeros(frame.shape,np.uint8)
    cv2.rectangle(rect,start_point,end_point,(255,255,255),2)
    rect = cv2.warpPerspective(rect,H[0],(frame.shape[1],frame.shape[0]))
    res[0:frame.shape[0],0:frame.shape[1],:] = cv2.add(frame,rect)
    res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
    for i in range(cur_corners.shape[0]):
      cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(pre_corners[i][0])+frame.shape[1],int(pre_corners[i][1])),(0,255,0),1)
    # print(index_delete)
    cv2.imshow("result",res)

In [205]:
def match_sift():
  global frame_copy2,f,rect,mask_ouput,init_corners,pre_frame,pre_corners
  if(f==0):
    pre_gray = cv2.cvtColor(frame_copy2,cv2.COLOR_RGB2GRAY)
    pre_gray = cv2.GaussianBlur(pre_gray,(7,7),0)
    sift = cv2.xfeatures2d_SIFT.create()  # 新建一个sift算子
    keyPoint, descriptor = sift.detectAndCompute(pre_gray, None)
    arr= []
    for i in keyPoint:
      if(i.pt[0]>=min_x and i.pt[0]<=min_x+width and i.pt[1]>=min_y and i.pt[1]<=min_y+height):
        arr.append(np.array([i.pt[0],i.pt[1]],np.float32))
    pre_corners = np.array(arr)
    init_corners = np.array(arr)
    f=1
  cur_corners,status,err = cv2.calcOpticalFlowPyrLK(pre_frame,frame,pre_corners,None)
  pre_frame = frame.copy()
  pre_corners = cur_corners.copy()
  tr_num = 0
  for i in status:
    if(i[0]==1):
      tr_num+=1
  # print(tr_num)
  if(tr_num >= 4):
    H = cv2.findHomography(pre_corners,cur_corners)
    res = np.zeros((frame.shape[0],frame.shape[1]*2,frame.shape[2]),"uint8")
    res[0:frame.shape[0],0:frame.shape[1],:] = frame
    res[0:frame.shape[0],frame.shape[1]:frame.shape[1]*2,:] = frame_copy1
    index_delete = []
    for i in range(cur_corners.shape[0]):
      if(cur_corners[i][0]<0 or cur_corners[i][0]>frame.shape[1] or cur_corners[i][1]<0 or cur_corners[i][1]>frame.shape[0]):
        index_delete.append(i)
        continue
      cv2.line(res,(int(cur_corners[i][0]),int(cur_corners[i][1])),(int(init_corners[i][0])+frame.shape[1],int(init_corners[i][1])),(0,255,0),1)
    # print(index_delete)
    cur_corners = np.delete(cur_corners,index_delete)
    cv2.imshow("result",res)

In [206]:
# match_sift()

In [207]:
# cap = cv2.VideoCapture(0)
# cv2.namedWindow('result')
# while(1):
#   ret,frame = cap.read()#摄像头读取，ret为是否程庚打开摄像头，frame是每一帧图像
#   frame = cv2.flip(frame, 1)#摄像头是和人对立的，将图像左右调换回来正常显示
#   if cv2.waitKey(1) & 0xFF == ord('q'):
#     break
#   match()
# cap.release()
# cv2.destroyAllWindows()

In [208]:
cap = cv2.VideoCapture(0)
# print(cap.set(5, 100))
print("读入视频的帧率为：",cap.get(cv2.CAP_PROP_FPS))
flag=0
start_flag = 0
cv2.namedWindow('get_target')
cv2.setMouseCallback('get_target',myMouse)
while(1):
  if(start_flag == 0):
    ret,frame = cap.read()#摄像头读取，ret为是否程庚打开摄像头，frame是每一帧图像
    frame = cv2.flip(frame, 1)#摄像头是和人对立的，将图像左右调换回来正常显示
  cv2.imshow("get_target",frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
  if(flag):break
cap.release()
cv2.destroyAllWindows()
cap = cv2.VideoCapture(0)
cv2.namedWindow('result')
f = 0
mask_ouput = None
pre_corners = None
pre_frame = frame_copy2
fps = []
while(1):
  ret,frame = cap.read()#摄像头读取，ret为是否成功打开摄像头，frame是每一帧图像
  frame = cv2.flip(frame, 1)#摄像头是和人对立的，将图像左右调换回来正常显示
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break
  # match0_orb()
  start = time.time()
  # match0_sift()
  match()
  # match2()
  # match2_sift()
  # match_sift()
  seconds = time.time()-start
  fps.append(1/seconds)
  #更新target
print("输出的平均帧率为：",np.average(fps))
cap.release()
cv2.destroyAllWindows()

True
读入视频的帧率为： 30.0
输出的平均帧率为： 90.93223552220253
